## Basic Working Chat with Memory

In [56]:
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage, BaseMessage
from langchain.memory import ConversationBufferWindowMemory
import streamlit as st
from fastapi import APIRouter
from typing import List

In [ ]:
class HTTPErrorResponse(BaseModel):
    detail: str

class InitiateChatResult(BaseModel):
    messages: List[str]

class PostChatResult(BaseModel):
    message: str

In [65]:
class BaseChatBot:
    router: APIRouter


    def _init_api_routes(self) -> None:
        self.router.add_api_route(
            "/initiate",
            endpoint=self.initiate_chat,
            methods=["POST"],
            responses={
                200: {"model": InitiateChatResult},
                401: {"model": HTTPErrorResponse},
                403: {"model": HTTPErrorResponse},
            },
        )

        self.router.add_api_route(
            "/",
            endpoint=self.get_prediction_with_ctx,
            methods=["POST"],
            responses={
                200: {"model": PostChatResult},
                400: {"model": HTTPErrorResponse},
                401: {"model": HTTPErrorResponse},
                403: {"model": HTTPErrorResponse},
            },
        )


    def __init__(self, ctx):
        self.ctx = ctx
        self.chat_history = []
        self.messages = []
        self._model = ChatOpenAI(
                        openai_api_key="NONE",  # type: ignore
                        openai_api_base="http://127.0.0.1:1234/v1",
                        max_tokens=800,
                        temperature=0.3,
                        streaming=False,
                    )
        with open("./model_server/prompts/chat.txt","r") as f:
            self._system_prompt = f.read()
        

    def get_prediction_with_ctx(self, message) -> str:
        
        self.chat_history.append(HumanMessage(content=message))
        messages:List[BaseMessage] = [SystemMessage(content=self._system_prompt.format(user_context=self.ctx))]  + self.chat_history  # type: ignore
        result = self._model.predict_messages( 
            messages=messages
        )
        self.chat_history.append(AIMessage(content=result.content))
        return result.content


    def initiate_chat(self):
        pass


In [66]:

ctx = "The user is Rohan, a student in 4th year B. Tech. CSE, and he needs help with Python programming"

model = BaseChatBot(ctx=ctx)


In [67]:
result = model.get_prediction_with_ctx(
    message="Hi, I want to know about the SHA algorithm. Can you give me an example?"
)

print(result)


Sure! The Secure Hash Algorithm (SHA) is a cryptographic hash function that was designed by the United States National Institute of Standards and Technology (NIST). It's used for data integrity checking and digital signatures, among other things. There are several versions of SHA, with SHA-256 being one of the most commonly used ones.

Here's an example: Let's say we want to hash the string "Hello World". We would use a hash function like SHA-256 to generate a fixed-size output (in this case, 256 bits) that represents the input data. The output of hashing "Hello World" using SHA-256 is:

`3104eab577c8f8d9b5a5e9d4e1a08b3e64a0edc0a06e10d7e1ba0e8b43e8b24f`

This output is unique to the input data and can be used for verifying the integrity of the data or creating digital signatures.


## RAG

In [60]:
result = model.get_prediction_with_ctx(
    message="Why is this better than basic hash? is there something special?"
)

print(result)


The SHA algorithm provides stronger security guarantees compared to basic hashing techniques such as MD5 or SHA-1. The main differences are:

1. Collision Resistance: Basic hashes like MD5 can be vulnerable to collisions, meaning two different inputs can produce the same hash value. This weakness can be exploited by attackers to forge signatures or tamper with data. On the other hand, SHA-2 and SHA-3 are designed to have a much higher resistance against collision attacks.
2. Security Strength: The security strength of SHA is regularly updated as new threats emerge. For example, NIST announced in 2015 that SHA-1 would be phased out by 2017 due to increasing vulnerabilities. In contrast, basic hashes like MD5 have remained unchanged for decades and are now considered insecure.
3. Standardization: SHA is a widely adopted standard developed by NIST, which ensures interoperability across different platforms and applications. This makes it easier to implement and verify the integrity of dat

In [61]:
result = model.get_prediction_with_ctx(
    message="Can you explain more?"
)

print(result)


Sure! Let's dive deeper into some key aspects of SHA:

1. Collision Resistance: SHA uses a mathematical function called the compression function that takes in the current state of the hash and combines it with the data block to produce a new state. This process is repeated for all blocks, and the final state is the hash value. The design of SHA ensures that finding two different inputs that produce the same output (a collision) is computationally infeasible. In contrast, basic hashes like MD5 can be vulnerable to collisions, which can be exploited by attackers to forge signatures or tamper with data.
2. Security Strength: SHA's security strength is regularly updated as new threats emerge. For example, NIST announced in 2015 that SHA-1 would be phased out by 2017 due to increasing vulnerabilities. In contrast, basic hashes like MD5 have remained unchanged for decades and are now considered insecure.
3. Standardization: SHA is a widely adopted standard developed by NIST, which ensures i

In [62]:
result = model.get_prediction_with_ctx(
    message="do you know DSS"
)

print(result)


Yes, I do! The Digital Signature Standard (DSS) is a United States government standard for digital signatures that uses the SHA algorithm in conjunction with elliptic curve cryptography (ECC). DSS is designed to provide secure and efficient digital signatures for electronic transactions.

The main components of DSS are:

1. Public Key: A public key, which is a mathematical function used to verify the authenticity of a signature. It is publicly available and can be used by anyone to check the validity of a signature.
2. Private Key: A private key, which is a secret value known only to the signer. It is used to generate digital signatures that can be verified using the public key.
3. Elliptic Curve Cryptography (ECC): ECC is a type of public-key cryptography that uses elliptic curves over finite fields as its mathematical basis. It provides similar security levels as traditional RSA-based systems but with shorter key sizes, making it more efficient and faster.
4. SHA Algorithm: DSS uses

In [63]:
result = model.get_prediction_with_ctx(
    message="Can you summarize all these so I can learn it for my exam tomorrow?"
)

print(result)


Sure! Here's a summary of the key points we discussed:

1. SHA (Secure Hash Algorithm) is a set of cryptographic hash functions designed by NIST, offering stronger security guarantees than basic hashing techniques like MD5 or SHA-1.
2. SHA provides higher collision resistance, more frequent updates for security strength, standardization across platforms, and flexibility in hash sizes compared to basic hashes.
3. DSS (Digital Signature Standard) is a United States government standard that uses the SHA algorithm in conjunction with elliptic curve cryptography (ECC) to provide secure and efficient digital signatures for electronic transactions.


In [64]:
for i in model.chat_history:
    print(i.content, "\n\n")

Hi, I want to know about the SHA algorithm. Can you give me an example? 



The Secure Hash Algorithm (SHA) is a set of cryptographic hash functions designed by the United States National Institute of Standards and Technology (NIST). It was initially published in 1993, and it has since been updated to several versions, including SHA-0, SHA-1, SHA-2, and SHA-3.

One common use of SHA is for digital signatures and message authentication codes (MACs), which help ensure the integrity of data transmitted over networks or stored on computers.

An example of using SHA to hash a string "Hello World" would be:

1. Convert the string into a series of binary digits (bits) using an encoding scheme such as ASCII or Unicode. For example, "Hello World" in ASCII is represented as 01001000 01101100 01101100 01101111 00100000 01110100 01101111 01110010 00100000 01101110 01101001 01100100 01101111 01110100 01100010 01101111 01110011.
2. Divide the bit string into fixed-size blocks, usually 512 bits for S